Imports

In [30]:
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import pprint
from gspread_dataframe import get_as_dataframe
import time
from datetime import date
import gspread

Connect to Google Sheets

In [31]:
service_account=gspread.service_account(filename="gambling-7-24-bf73ca0313d0.json")

In [32]:
sheet=service_account.open("Discipline")

In [33]:
worksheet=sheet.worksheet("Picks")

In [41]:
df = pd.DataFrame(worksheet.get_all_records())

Dataframe Calculations

###### Running Total

In [52]:
df['Win/Lose']=df["Win/Lose"].replace({'\$':''}, regex = True)
df['Win/Lose']=pd.to_numeric(df["Win/Lose"])
running_total=df['Win/Lose'].sum()